In [34]:
def transformar_periodo(periodo):
    # Diccionario para mapear los códigos de mes a números de mes
    meses = {
        'en': '01',
        'fe': '02',
        'ma': '03',
        'ab': '04',
        'my': '05',
        'jn': '06',
        'jl': '07',
        'ag': '08',
        'se': '09',
        'oc': '10',
        'no': '11',
        'di': '12',
    }
    mes_codigo = periodo[:2]
    anio = periodo[2:]
    mes = meses.get(mes_codigo, '01')  # Por defecto enero si no se encuentra el mes
    return pd.to_datetime(f'01/{mes}/{anio}', format='%d/%m/%Y')

def leer_transform_xls(ruta):
    df = pd.read_excel(ruta, skiprows=8, header=None)
    df.dropna(how='all', inplace=True)
    df = df.iloc[:,:5] #nos aseguramos tomar solo las columnas iniciales, en caso existan columnas auxiliares
    return df

def estructurar_xls(df, nombre_libro, nombres_columas):
    tipo_entidad = nombre_libro[:6]

    periodo = nombre_libro.split('.')[0]
    periodo = periodo[-6:] #llamar a una función que convierta periodo a formato fecha

    periodo = transformar_periodo(periodo)

    #renombramos las columnas
    try:
        df.columns = nombres_columas

        #hacerlo dinámico
        df.insert(0, 'tipo_entidad', tipo_entidad)
        df.insert(1, 'periodo', periodo)
        return df
    except:
        col_aux = nombres_columas.copy()
        #quitar 'nivel_3' de la lista
        col_aux.pop(3)
        estructurar_xls(df, nombre_libro, col_aux)
        return df


In [35]:
import pandas as pd
import os


#leer_data y retornar un df csv

carpeta_lectura = 'Patrimonio_Efectivo'
carpeta_lectura = os.path.join('../', 'descargas',carpeta_lectura)
libros = os.listdir(carpeta_lectura)

#filtrar solo a los .XLS
libros = [i for i in libros if i.endswith('.XLS')]

nombres_columas = ['entidad','nivel_1','nivel_2','nivel_3', 'total']




In [36]:
df_total = pd.DataFrame()

for libro in libros:

    try:

        ruta = os.path.join(carpeta_lectura, libro)

        #leemos el libro XLS con pequeños cambios
        df_bruto = leer_transform_xls(ruta)

        #estructuramos las columnas
        df = estructurar_xls(df_bruto, libro, nombres_columas)

        #concateno todos los libros a uno solo
        df_total = pd.concat([df_total, df], ignore_index=True)
    except Exception as e:
        print(f"No se pudo procesar para el libro: {libro}. Error: {e}")


In [37]:
#solo para esta columna completamos con 0 porque algunos periodos cambian haciendo que nivel_3 no existe una temporada
df_total['nivel_3'] = df_total['nivel_3'].fillna(0)


#eliminamos filas con valores nulos
df_total.dropna(axis=0,inplace=True)

carpeta_salida = os.path.join(carpeta_lectura, "_consolidado")

os.makedirs(carpeta_salida, exist_ok=True)
archivo_salida = os.path.join(carpeta_salida, "consolidado.xlsx")

#convertimos a número columnas tipo object
columnas_a_convertir = ["nivel_1", "nivel_2", "nivel_3", "total"]
for col in columnas_a_convertir:
    df_total[col] = pd.to_numeric(df_total[col], errors="coerce")


#guardamos
df_total.to_excel(archivo_salida, index=False)

In [39]:
#convertimos a número columnas tipo object
columnas_a_convertir = ["nivel_1", "nivel_2", "nivel_3", "total"]
for col in columnas_a_convertir:
    df_total[col] = pd.to_numeric(df_total[col], errors="coerce")

df_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 834 entries, 1 to 958
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   tipo_entidad  834 non-null    object        
 1   periodo       834 non-null    datetime64[ns]
 2   entidad       834 non-null    object        
 3   nivel_1       834 non-null    float64       
 4   nivel_2       834 non-null    float64       
 5   nivel_3       834 non-null    float64       
 6   total         834 non-null    float64       
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 52.1+ KB


In [44]:
#comprobacion

df_total['nivel_1_%'] = (df_total['nivel_1']/100) * df_total['total']
df_total['nivel_2_%'] = (df_total['nivel_2']/100) * df_total['total']
df_total['nivel_3_%'] = (df_total['nivel_3']/100) * df_total['total']

df_total['total_%'] = df_total['nivel_1_%'] + df_total['nivel_2_%'] +df_total['nivel_3_%']

df_total['diff'] = df_total['total'] - df_total['total_%']

df_total[['diff']].describe()


,diff
count,8.340000e+02
mean,3.681600e-12
std,2.191881e-10
min,-1.862645e-09
25%,0.000000e+00
50%,0.000000e+00
75%,0.000000e+00
max,1.862645e-09
